In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
def load_embedding(filename='embedding.csv'):
    return pd.read_csv(filename, header=None)

In [5]:
emb = load_embedding()

In [6]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.363037,0.334529,-0.165212,-0.409537,-0.298907,-0.652681,0.433844,0.339051,-0.419195,0.162223,...,-0.150816,0.380193,0.763373,-0.686894,-0.139185,-0.273024,0.187815,0.011107,0.785204,-0.255765
1,-0.584020,0.390310,0.652820,-0.340300,0.194930,-0.834890,0.119290,-0.572910,-0.568440,0.729890,...,0.285860,-0.052162,-0.508180,-0.634590,0.338890,0.284160,-0.203400,-1.233800,0.467150,0.788580
2,0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
3,-0.039369,1.203600,0.354010,-0.559990,-0.520780,-0.669880,-0.754170,-0.653400,-0.232460,0.586860,...,-0.601410,0.504030,-0.083316,0.202390,0.443000,-0.060769,-0.428070,-0.084135,0.491640,0.085654
4,-0.268200,-0.478700,0.180990,-0.538370,-0.240210,-0.562030,0.209440,0.423580,-0.461470,0.769030,...,0.005399,-0.533440,-0.187060,0.522540,0.243610,0.051387,0.272100,-0.444330,0.019464,0.627820


In [7]:
emb_dim = emb.shape[1]

In [8]:
emb_dim

50

In [9]:
def load_dict(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def load_index_word_map(word2ind_filename='word2ind', ind2word_filename='ind2word'):
    word2ind = load_dict(word2ind_filename)
    ind2word = load_dict(ind2word_filename)
    return word2ind, ind2word

In [10]:
word2ind, ind2word = load_index_word_map()

In [11]:
vocab_size = len(word2ind)

In [12]:
def load_training_samples(filename='train.csv'):
    return pd.read_csv(filename, header=None)

In [13]:
dataset = load_training_samples()

In [14]:
def train_valid_split(x, y, test_size=0.3, random_state=43):
    assert len(x) == len(y), 'Feature and label must have same length.'
    np.random.RandomState(seed=random_state)
    length = len(x)
    choices = list(range(length))
    val_choices = np.random.choice(choices, int(length*test_size), replace=False).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return x[train_choices, :], x[val_choices, :], y[train_choices, :], y[val_choices, :]

In [15]:
train_x, val_x, train_y, val_y = train_valid_split(dataset.values[:, :-1], dataset.values[:, -1:], test_size=0.3, random_state=43)

In [16]:
train_x.shape

(10972446, 20)

In [17]:
train_y.shape

(10972446, 1)

In [18]:
val_x.shape

(4702476, 20)

In [19]:
val_y.shape

(4702476, 1)

In [20]:
emb.shape

(81452, 50)

In [21]:
input_length = train_x.shape[1]

In [22]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, weights=[emb]))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(vocab_size, activation='softmax'))

In [22]:
model = load_model('model20171230_6.h5')

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            4072600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
__________

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1, batch_size=1000)

Train on 10972446 samples, validate on 4702476 samples
Epoch 1/1
10972446/10972446 [==============================] - 2716s - loss: 4.8483 - acc: 0.2186 - val_loss: 4.3978 - val_acc: 0.2574


In [27]:
# save model
model.save('model20180117_1.h5')

In [28]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1, batch_size=1000)

Train on 10972446 samples, validate on 4702476 samples
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[5000,81452]
	 [[Node: training/Adam/gradients/loss/dense_2_loss/clip_by_value_grad/GreaterEqual = GreaterEqual[T=DT_FLOAT, _class=["loc:@loss/dense_2_loss/clip_by_value"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/dense_2_loss/clip_by_value/Minimum, loss/dense_2_loss/Const)]]
	 [[Node: metrics/acc/Mean/_53 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_5758_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'training/Adam/gradients/loss/dense_2_loss/clip_by_value_grad/GreaterEqual', defined at:
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/fei/miniconda3/envs/py3_5/lib/python3.5/runpy.py", line 85, in _ru

In [27]:
# save model
model.save('model20180117_2.h5')